## 文本分类
作用：识别垃圾邮件、情感分类、主题分类等

分类问题：模型、预测、学习
模型：分类器。分类器是一个函数f，这个函数拿到输入x然后输出类别y

### 情感分析
在这份notebook中，我们会用PyTorch模型和TorchText再来做情感分析（检测一段文字的情感是正面的还是负面的）。采用IMDb数据集，即电影评论。

模型从简单到复杂，会依次构建：
- Word Averaging模型
- RNN/LSTM模型
- CNN模型

### 准备数据
- TorchText中的一个重要概念是Field，Field决定了你的数据会被怎样处理。在我们的情感分类任务中，我们所需要接触到的数据有文本字符串和两种情感，“pos”或者“neg”。
- Field的残念书制定了数据会被怎样处理。
- 我们使用TEXT field来定义如何处理电影评论，使用LABEL field来处理两个情感类别。
- 我们的TEXT field带有tokenize = 'spacy',这表示会用spaCy tokenizer来tokenize英文句子。如果不特别声明tokenize这个参数，那么默认的分词方法是用空格。
- LABEL由LabelField定义，这是一种特别的用来处理label的Field。

In [1]:
import spacy
spacy.load('en_core_web_sm')

In [2]:
from spacy.lang.en import English


In [3]:
import torch
from torchtext import data

import warnings
warnings.filterwarnings("ignore")

SEED = 1234

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize='spacy',tokenizer_language='en_core_web_sm')
LABEL = data.LabelField(dtype=torch.float)

- TorchText支持很多常见的自然语言处理数据集。
- 下面的代码会自动下载IMDb数据集，然后分成train/test两个torchtext.datasets类别，数据被前面的Fields处理。IMDb一共有50000电影评论，每个评论都被标注为正面的或负面的。

In [4]:
from torchtext import datasets
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

查看每个数据split有多少条数据

In [5]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')


Number of training examples: 25000
Number of testing examples: 25000


查看一个example

In [6]:
print(vars(train_data.examples[0]))

{'text': ['Bromwell', 'High', 'is', 'a', 'cartoon', 'comedy', '.', 'It', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life', ',', 'such', 'as', '"', 'Teachers', '"', '.', 'My', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'Bromwell', 'High', "'s", 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', '"', 'Teachers', '"', '.', 'The', 'scramble', 'to', 'survive', 'financially', ',', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', 'teachers', "'", 'pomp', ',', 'the', 'pettiness', 'of', 'the', 'whole', 'situation', ',', 'all', 'remind', 'me', 'of', 'the', 'schools', 'I', 'knew', 'and', 'their', 'students', '.', 'When', 'I', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school', ',', 'I', 'immediately', 'recalled', '.........', 'at', '..........', 'High', '.', 'A', 'classic', 'l

- 由于我们现在只有train/test这两个分类，所以我们需要创建一个新的validation set，可以使用.split()来创建新的分类。
- 默认的数据分割是70，30，如果声明split_ratio，可以改变split之间的比例，split_ratio=0.8表示80%的数据是训练集，20%是验证集。
- 我们还声明random_state这个参数，确保我们每次分割的数据集都是一样的。

In [7]:
import random
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [8]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')


Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


- 下一步需要创建vocabulary，vocabulary就是把每个单词一一映射到一个数字

word|index|one-hot vector
:--:|:--:|:--:
I|0|[1,0,0,0]
hate|1|[0,1,0,0]
this|2|[0,0,1,0]
film|3|[0,0,0,1]

- 采用最常见的25k个单词来构建单词表，用max_size参数来实现这一点
- 其他的单词用`<unk>`来表示

In [9]:
# glove是预训练的意思，把预训练的100维的向量拿进来
# glove是斯坦福训练的比较高质量的词向量,把词向量初试为glove收敛速度会快很多
TEXT.build_vocab(train_data, max_size = 25000, vectors = "glove.6B.100d", unk_init=torch.Tensor.normal_)
LABEL.build_vocab(train_data)

In [10]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


当我们把好几个句子传进模型的时候，我们是按照一个个batch传进去的，也就是说，我们一次传入了好几个句子，而且每个batch中的句子必须是相同的长度。为了确保句子的长度相同，TorchText会把短的句子pad到和最长的句子等长。

sent1|sent2
:--:|:--:
I|This
hate|film
this|sucks
film|`<pad>`

下面我们来看训练数据集中最常见的单词

In [11]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 203566), (',', 192495), ('.', 165612), ('and', 109442), ('a', 109116), ('of', 100702), ('to', 93766), ('is', 76328), ('in', 61255), ('I', 54004), ('it', 53508), ('that', 49187), ('"', 44282), ("'s", 43329), ('this', 42445), ('-', 36690), ('/><br', 35752), ('was', 35034), ('as', 30384), ('with', 29774)]


或者采用stoi(string to int)或者iots(int to string)来查看单词表


In [12]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


查看labels

In [13]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


- 最后一步数据的准备是创建iterators。每个iteration都会返回一个batch的examples。
- 我们会使用BucketIterator。BucketIterator会把长度差不多的句子放到同一个batch中，确保每个batch中不出现太多的padding
- 严格来说，我们这份notebook中的模型代码都有一个问题，也就是我们把`<pad>`也当作了模型的输入进行训练，更好的做法是在模型中把由`<pad>`产生的输出给消除掉。
- 如果有GPU，还可以指定每个iteration返回的tensor都在GPU上

In [14]:
BATCH_SIZE = 8

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device)

In [15]:
# 一列是一个句子
batch = next(iter(valid_iterator))

### CNN-1

#### 来源
- 需求需要：自编码器（多层全接连神经网络）计算太过于复杂，网络参数众多
- 视觉上的可行性：图像上的某一部分像素所表示的意义其实只和它周围的像素领域有关。
![](images2\37.png)

#### 关于卷积
[卷积](https://blog.csdn.net/bitcarmanlee/article/details/54729807)

在CNN中，基本都是与小模版进行的操作，我们需要设计的不过是模板的大小、个数这些，至于是什么样的模板，就是靠自己学习而来的。

#### 关于卷积层
我们可以定义好几个模板，分别依次卷积：
![](images\38.png)

模板之间不同，模板中的值在CNN中就是需要学习的权值参数，而不是自己定义的。一副图像经过与好多不同的模板卷积就会得到一些所谓的特征MAP，在得到这些特征MAP后，其实就可以发现这些MAP本质上也是图像，那么如果对这些图像再进行后续的卷积，就会自然而然的引出深层的卷积网络。

![](images2\39.png)

在这里呢我们假设卷积层C1下的每一个MAP又会衍生两个C2下的MAP，就是上面那样。但是在实际的CNN中，为了进一步减少参数，优化模型，通常早卷积层C1与C2之间还有一个降采样层S，什么意思呢？最简单就是对每一个C1下MAP进行缩小吧，比如缩小一倍，就是让C1下的每个MAP中每`2*2`的窗口取一个平均像素变成一个像素，这样就达到了缩小的目的了，这也就是降采样。像下面这样子。

![](images2\40.png)


之后可以再对C2进行降采样，在进行卷积，一直下去就是深度CNN了。

还有一个问题就是，我们从S1到C2的时候，我们是选择S1中的一个map卷积得到下一个MAP，其实我们完全可以用S1中的两个甚至全部的3个MAP进行组合得到下一个MAP，组合方式可以是直接相加，比如说C2中每一个MAP都是S1中的所有MAP进行3个模版卷积然后相加而来的，那么此时的图就是下面这样子： 

![](images2\41.png)

C2中的每一个MAP由S1中的3个MAP分别进行一个卷积，再把他们相加起来组合成一个MAP，注意的是从S1到C2的每一个卷积模版都是不一样的。那么想必前面一种可以看到，以前从S1到C2可能只要6个滤波模版，而现在呢需要18个模版，每个模版都是需要学习出来的。当然18个是最多的，如果后面每一个MAP不与前面的3个都相连接，而是2个或者1个，就不需要18个了。并且后一个MAP中每个与前面几个相连也不一定非得相同，比如C2中MAP1可能与S1的MAP1相连接，而C2中MAP2可能与S1的MAP1与MAP2相连接，等等。

### CNN-2

#### 结构
![](images2\42.png)
 根据上图 卷积神经网络 ConvNet 可以分为4大层：

1. 图像输入Image Input：为了减小后续BP算法处理的复杂度，一般建议使用灰度图像。也可以使用RGB彩色图像，此时输入图像原始图像的RGB三通道。对于输入的图像像素分量为 [0, 255]，为了计算方便一般需要归一化，如果使用sigmoid激活函数，则归一化到[0, 1]，如果使用tanh激活函数，则归一化到[-1, 1]。

2. 卷积层(Convolution Layer)：特征提取层(C层) - 特征映射层(S层)。将上一层的输出图像与本层卷积核(权重参数w)加权值，加偏置，通过一个Sigmoid函数得到各个C层，然后下采样subsampling得到各个S层。C层和S层的输出称为Feature Map(特征图)。

3. 光栅化(Rasterization)：为了与传统的多层感知器MLP全连接，把上一层的所有Feature Map的每个像素依次展开，排成一列。

4. 多层感知器(MLP)：最后一层为分类器，一般使用Softmax，如果是二分类，当然也可以使用线性回归Logistic Regression，SVM，RBM。

详细地展开2卷积层：

- C层：特征提取层。每个神经元的输入与前一层的局部接受域相连，并提取该局部的特征。一旦该局部特征被提取后，它与其它特征间的位置关系也随之确定下来。

- S层：特征映射层。网络的每个计算层由多个特征映射组成，每个特征映射为一个平面，平面上所有神经元的权值相等。特征映射结构采用影响函数核小的sigmoid函数作为卷积网络的激活函数，使得特征映射具有位移不变性。此外，由于一个映射面上的神经元共享权值，因而减少了网络自由参数的个数，降低了网络参数选择的复杂度。卷积神经网络中的每一个特征提取层（C层）都紧跟着一个用来求局部平均与二次提取的计算层（S层），这种特有的两次特征提取结构使网络在识别时对输入样本有较高的畸变容忍能力。

####  CNN三大核心思想

卷积神经网络CNN的出现是为了解决MLP多层感知器全连接和梯度发散的问题。其引入三个核心思想：1.局部感知(local field)，2.权值共享(Shared Weights)，3.下采样(subsampling)。极大地提升了计算速度，减少了连接数量。

**1.局部感知**

形象地说，就是模仿你的眼睛，人看东西的时候，目光是聚焦在一个相对很小的局部。严格一些说，普通的多层感知器中，隐层节点会全连接到一个图像的每个像素点上，而在卷积神经网络中，每个隐层节点只连接到图像某个足够小局部的像素点上，从而大大减少需要训练的权值参数。

对于一个 1000∗1000 的输入图像而言，如果下一个隐藏层的神经元数目为 106 个，采用全连接则有 1000∗1000∗106=1012 个权值参数，如此数目巨大的参数几乎难以训练；而采用局部连接，隐藏层的每个神经元仅与图像中 10∗10的局部图像相连接，那么此时的权值参数数量为 10∗10∗106=108，将直接减少4个数量级。

如下图所示，左边是每个像素的全连接，右边是每行隔两个像素作为局部连接，因此在数量上，少了很多权值参数数量每一条连接每一条线需要有一个权值参数。
![](images2\43.png)

局部感知如何使 全连接变成局部连接，按照人工神经网络的方法，把输入图像的像素一字排开之后，每一个像素值就是一个神经元输入，需要对隐层或者输出层做全连接，如上图左侧所示。卷积神经网络引入卷积概念后，卷积核通过原图像，然后卷积核对原图像上符合卷积核大小的像素进行加权求和，每一次只是对符合卷积核的图像像素做卷积，这就是局部感知的概念，使 全连接变成局部连接。

![](images2\44.png)

**2.权值共享**

尽管局部感知使计算量减少了几个数量级，但权重参数数量依然很多。能不能再进一步减少呢？方法就是权值共享。

权值共享：不同的图像或者同一张图像共用一个卷积核，减少重复的卷积核。同一张图像当中可能会出现相同的特征，共享卷积核能够进一步减少权值参数。

如下图所示，为了找到鸟嘴，一个激活函数A需要检测图像左侧有没有鸟嘴，另外一个激活函数B需要检测另外一张图像中间有没有类似的鸟嘴。其实，鸟嘴都可能具有同样的特征，只需要一个激活函数C就可以了，这个时候，就可以共享同样的权值参数（也就是卷积核）。

![](images2\45.png)

如果使用了权值共享（共同使用一个卷积核），那么将可以大大减少卷积核的数量，加快运算速度。

![](images2\46.png)

 举个栗子，在局部连接中隐藏层的每一个神经元连接的是一个 10∗10 的局部图像，因此有 10∗10 个权值参数，将这 10∗10 个权值参数共享给剩下的神经元，也就是说隐藏层中 106 个神经元的权值参数相同，那么此时不管隐藏层神经元的数目是多少，需要训练的参数就是这 10∗10 个权值参数（也就是卷积核(也称滤波器)的大小。

尽管只有这么少的参数，依旧有出色的性能。但是，这样仅提取了图像的一种特征，如果要多提取出一些特征，需要增加多个卷积核，不同的卷积核能够得到图像的不同映射下的特征，称之为 Feature Map。如果有100个卷积核，最终的权值参数也仅为 100∗100=$10^4$ 个而已。另外，偏置参数b也是共享的，同一种滤波器共享一个。

**3.池化**

 在卷积神经网络中，没有必要一定就要对原图像做处理，而是可以使用某种“压缩”方法，这就是池化，也就是每次将原图像卷积后，都通过一个下采样的过程，来减小图像的规模。

pooling的好处有什么？
1. 这些统计特征能够有更低的维度，减少计算量。
2. 不容易过拟合，当参数过多的时候很容易造成过度拟合。
3. 缩小图像的规模，提升计算速度。

如下图所示，原图是一张500∗500 的图像，经过subsampling之后哦，变成了一张 250∗250 的图像。这样操作的好处非常明显，虽然经过权值共享和局部连接后的图像权值参数已经大大减少，但是对于计算量来说，还是非常巨大，需要消费很大的计算时间，于是为了进一步减少计算量，于是加入了subsampling这个概念，不仅仅使图像像素减少了， 同时也减少计算时间。

![](images2\47.png)

以最大池化（Max Pooling）为例，1000×1000的图像经过10×10的卷积核卷积后，得到的是991×991的特征图，然后使用2×2的池化规模，即每4个点组成的小方块中，取最大的一个作为输出，最终得到的是496×496大小的特征图。

下采样，即池化，目的是减小特征图，池化规模一般为2×2。常用的池化方法有：

Pooling算法

最大池化（Max Pooling）。取4个点的最大值。这是最常用的池化方法。
均值池化（Mean Pooling）。取4个点的均值。
可训练池化。训练函数 f ，接受4个点为输入，出入1个点。

由于特征图的变长不一定是2的倍数，所以在边缘处理上也有两种方案：

保留边缘。将特征图的变长用0填充为2的倍数，然后再池化。
忽略边缘。将多出来的边缘直接省去。

#### CNN物理意义

 了解三个关于CNN的核心特性之后，来看看CNN的具体是怎么运作的。

为了从原始图像得到C层，需要把原始图像中的每一个像素值作为神经网络当中一个神经元，那么这里把原始输入图像一字排开，作为输入层。通过BP反向传播算法计算好的权值参数（卷积核）去计算C层对应的的每一个像素的值。

![](images2\48.png)

从上图我们得到了C层，也就是提取特征后得到特征层，需要对特征层处理减少特征数量，进一步抽取高层特性，因此需要进步特征映射层（S层）。下图的pooling层（S层）使用了max pooling算法，pooling核为2x2，没有重叠部分，取每4个像素中最大的一个像素值作为新的像素值。

![](images2\49.png)

那么在这个模型当中，我们已经确定了激活函数φ(∗)，输入x1,x2,...,xn 是确定的，未知量就剩下神经元k的突触权值wk1,wk2,...,wkn ，bk 偏置。反向传播算法（back propagation）就是为了求整个神经网络当中的两种未知变量：权值 w 和偏置 b。在上图这个模型当中，卷积核大小为3∗3，也就是有9个权值w组成，因此反向传播的时候就是要求这两个卷积核的权值w，使用大量的图片作为输入就是为了使用BP算法求得卷积核的值，当求得卷积核的值之后，分类的时候输入一张未知的图片，然后通过整个网络，直接就可以得到最终的分类结果，因为权值和偏置已经通过训练求出来了，整个网络没有未知量。


### CNN-3

**1、卷积神经网络之层级结构**

![](images2\50.png)

1. 最左边
- 最左边是数据输入层，对数据做一些处理，比如去均值（把输入数据各个维度都中心化为0，避免数据过多偏差，影响训练效果）、归一化（把所有的数据都归一到同样的范围）、PCA/白化等等。CNN只对训练集做“去均值”这一步。

2. 中间
- CONV：卷积计算层，线性乘积求和。
- RELU：激励层。
- POOL：池化层，简言之，即取区域平均或最大

3. 最右边
- FC：全连接层

**2.CNN之卷积计算层**

1. CNN怎么进行识别

简言之，当我们给定一个"X"的图案，计算机怎么识别这个图案就是“X”呢？一个可能的办法就是计算机存储一张标准的“X”图案，然后把需要识别的未知图案跟标准"X"图案进行比对，如果二者一致，则判定未知图案即是一个"X"图案。

而且即便未知图案可能有一些平移或稍稍变形，依然能辨别出它是一个X图案。如此，CNN是把未知图案和标准X图案一个局部一个局部的对比，如下图所示

![](images2\51.png)

而未知图案的局部和标准X图案的局部一个一个比对时的计算过程，便是卷积操作。卷积计算结果为1表示匹配，否则不匹配。

具体而言，为了确定一幅图像是包含有"X"还是"O"，相当于我们需要判断它是否含有"X"或者"O"，并且假设必须两者选其一，不是"X"就是"O"。

![](images2\52.png)

理想的情况就像下面这个样子：

![](images\53.png)

标准的"X"和"O"，字母位于图像的正中央，并且比例合适，无变形

对于计算机来说，只要图像稍稍有一点变化，不是标准的，那么要解决这个问题还是不是那么容易的。

计算机要解决上面这个问题，一个比较天真的做法就是先保存一张"X"和"O"的标准图像（就像前面给出的例子），然后将其他的新给出的图像来和这两张标准图像进行对比，看看到底和哪一张图更匹配，就判断为哪个字母。

但是这么做的话，其实是非常不可靠的，因为计算机还是比较死板的。在计算机的“视觉”中，一幅图看起来就像是一个二维的像素数组（可以想象成一个棋盘），每一个位置对应一个数字。在我们这个例子当中，像素值"1"代表白色，像素值"-1"代表黑色。

![](images2\54.png)

当比较两幅图的时候，如果有任何一个像素值不匹配，那么这两幅图就不匹配，至少对于计算机来说是这样的。

对于这个例子，计算机认为上述两幅图中的白色像素除了中间的3x3的小方格里面是相同的，其他四个角上都不同，因此，从表面上看，计算机判别右边那幅图不是"X"，两幅图不同。

但是这么做，显得太不合理了。理想的情况下，我们希望，对于那些仅仅只是做了一些像平移，缩放，旋转，微变形等简单变换的图像，计算机仍然能够识别出图中的"X"和"O"。就像下面这些情况，我们希望计算机依然能够很快并且很准的识别出来：

![](images2\55.png)

这也就是CNN出现所要解决的问题。

![](images2\56.png)

对于CNN来说，它是一块一块地来进行比对。它拿来比对的这个“小块”我们称之为Features（特征）。在两幅图中大致相同的位置找到一些粗糙的特征进行匹配，CNN能够更好的看到两幅图的相似性，相比起传统的整幅图逐一比对的方法。

每一个feature就像是一个小图（就是一个比较小的有值的二维数组）。不同的Feature匹配图像中不同的特征。在字母"X"的例子中，那些由对角线和交叉线组成的features基本上能够识别出大多数"X"所具有的重要特征。

![](images2\57.png)

看到这里是不是有了一点头目呢。但其实这只是第一步，你知道了这些Features是怎么在原图上面进行匹配的。但是你还不知道在这里面究竟进行的是怎样的数学计算，比如这个下面3x3的小块到底干了什么？

![](images2\58.png)

这里面的数学操作，就是我们常说的“卷积”操作。

2. 什么是卷积

对图像（不同的数据窗口数据）和滤波矩阵（一组固定的权重：因为每个神经元的多个权重固定，所以又可以看做一个恒定的滤波器filter）做内积（逐个元素相乘再求和）的操作就是所谓的『卷积』操作，也是卷积神经网络的名字来源。

非严格意义上来讲，下图中红框框起来的部分便可以理解为一个滤波器，即带着一组固定权重的神经元。多个滤波器叠加便成了卷积层。

![](images2\59.png)

比如下图中，图中左边部分是原始输入数据，图中中间部分是滤波器filter，图中右边是输出的新的二维数据。

![](images2\60.png)

中间滤波器filter与数据窗口做内积，其具体计算过程则是：4x0 + 0x0 + 0x0 + 0x0 + 0x1 + 0x1 + 0x0 + 0x1 + -4x2 = -8

**3. 图像上的卷积**

在下图对应的计算过程中，输入是一定区域大小(widthxheight)的数据，和滤波器filter（带着一组固定权重的神经元）做内积后等到新的二维数据。

具体来说，左边是图像输入，中间部分就是滤波器filter（带着一组固定权重的神经元），不同的滤波器filter会得到不同的输出数据，比如颜色深浅、轮廓。相当于如果想提取图像的不同特征，则用不同的滤波器filter，提取想要的关于图像的特定信息：颜色深浅或轮廓。

![](images2\61.png)

**4. 动态分解卷积图**

在CNN中，滤波器filter（带着一组固定权重的神经元）对局部输入数据进行卷积计算。每计算完一个数据窗口内的局部数据后，数据窗口不断平移滑动，直到计算完所有数据。这个过程中，有这么几个参数： 

- 深度depth：神经元个数，决定输出的depth厚度。同时代表滤波器个数。
- 步长stride：决定滑动多少步可以到边缘。
- 填充值zero-padding：在外围边缘补充若干圈0，方便从初始位置以步长为单位可以刚好滑倒末尾位置，通俗地讲就是为了总长能被步长整除。 

![](images2\62.png)

动态卷积图：

![](images2\63.gif)

- 两个神经元，即depth=2，意味着有两个滤波器。
- 数据窗口每次移动两个步长取3*3的局部数据，即stride=2。
- zero-padding=1。

然后分别以两个滤波器filter为轴滑动数组进行卷积计算，得到两组不同的结果。

- 左边是输入（7x7x3中，7x7代表图像的像素/长宽，3代表R、G、B 三个颜色通道）
- 中间部分是两个不同的滤波器Filter w0、Filter w1
- 最右边则是两个不同的输出

随着左边数据窗口的平移滑动，滤波器Filter w0 / Filter w1对不同的局部数据进行卷积计算。

值得一提的是：左边数据在变化，每次滤波器都是针对某一局部的数据窗口进行卷积，这就是所谓的CNN中的**局部感知机制**。

- 打个比方，滤波器就像一双眼睛，人类视角有限，一眼望去，只能看到这世界的局部。如果一眼就看到全世界，你会累死，而且一下子接受全世界所有信息，你大脑接收不过来。当然，即便是看局部，针对局部里的信息人类双眼也是有偏重、偏好的。比如看美女，对脸、胸、腿是重点关注，所以这3个输入的权重相对较大。

与此同时，数据窗口滑动，导致输入在变化，但中间滤波器Filter w0的权重（即每个神经元连接数据窗口的权重）是固定不变的，这个权重不变即所谓的CNN中的参数（权重）共享机制。

- 再打个比方，某人环游全世界，所看到的信息在变，但采集信息的双眼不变。btw，不同人的双眼 看同一个局部信息 所感受到的不同，即一千个读者有一千个哈姆雷特，所以不同的滤波器 就像不同的双眼，不同的人有着不同的反馈结果。

分解如下：

![](images2\64.png)

上图中的输出结果1具体是怎么计算得到的呢？其实，类似wx + b，w对应滤波器Filter w0，x对应不同的数据窗口，b对应Bias b0，相当于滤波器Filter w0与一个个数据窗口相乘再求和后，最后加上Bias b0得到输出结果1，如下过程所示：

![](images2\65.png)

$（1*0 + 1*0 + -1*0）+（-1*0 + 0*0 + 1*1） +（-1*0 + -1*0 + 0*1）+$

![](images2\66.png)

$(-1*0 + 0*0 + -1*0) + (0*0 + 0*1 + -1*1) + (1*0 + -1*0 + 0*2) + $

![](images2\67.png)

$(0*0 + 1*0 + 0*0) + (1*0 + 0*2 + 1*0) + (0*0 + -1*0 + 1*0) + $

$1$

= 1

然后滤波器Filter w0固定不变，数据窗口向右移动2步，继续做内积计算，得到0的输出结果

![](images2\68.png)

最后，换做另外一个不同的滤波器Filter w1、不同的偏置Bias b1，再跟图中最左边的数据窗口做卷积，可得到另外一个不同的输出。

![](images2\69.png)


**5.CNN之激励层与池化层**

1. ReLU激励层

实际梯度下降中，sigmoid容易饱和、造成终止梯度传递，且没有0中心化。咋办呢，可以尝试另外一个激活函数：ReLU，其图形表示如下：

![](images2\70.png)

2. 池化pool层

池化，简言之，即取区域平均或最大，如下图所示

![](images2\71.png)

上图所展示的是取区域最大，即上图左边部分中 左上角2x2的矩阵中6最大，右上角2x2的矩阵中8最大，左下角2x2的矩阵中3最大，右下角2x2的矩阵中4最大，所以得到上图右边部分的结果：6 8 3 4。

### Word Aberaging模型
- 我们首先介绍一个简单的Word Averaging模型。这个模型非常简单，我们把每个单词都通过Embedding层投射成word embedding vecor，然后把一句话中的所有word vector做个平均，就是整个句子的vector表示了。接下来把这个sentence vector传入一个Linear层，做分类即可。
![](images2\35.png)

- 我们使用avg_pool2d来做average pooling，我们的目标是把sentence length那个维度平均成1，然后保留embedding这个维度。

![](images2\36.png)

- avg_pool2d的kernel size是(embedded.shape[1], 1),所以句子长度的那个维度会被压扁。

![](images2\72.png)

### 关于维度和squeeze和unsqueeze

维度：第一个数前面的中括号数量。拿到一个维度很高的向量，将最外层的中括号去掉，数最外层逗号的个数，逗号个数加一就是最高维度的维数，如此循环，直到全部解析完毕。

[维度](https://blog.csdn.net/wwwlyj123321/article/details/88972717)

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class WordAVGModel(nn.Module):
    def __init__(self, vocab_size, embedding_size, output_size, pad_idx):
        super(WordAVGModel, self).__init__()
        self.embed = nn.Embedding(vocab_size, embedding_size, padding_idx = pad_idx)
        self.linear = nn.Linear(embedding_size, output_size)
        
        
        
        
    def forward(self, text):
        # text是一个二维的数据，一列是一个句子
        embedded = self.embed(text) # [seq_len, batch_size, embedding_size]
       # embedded = embedded.transpose(1,0) # [batch_size, seq_len,  embedding_size]
        embedded = embedded.permute(1,0,2) # [batch_size, seq_len,  embedding_size]
        # 把seq_len压扁,对逐列求平均
        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze() # # [batch_size, 1,  embedding_size]
        
        return self.linear(pooled)
        
        

In [17]:
VOCAB_SIZE = len(TEXT.vocab)
EMBEDDING_SIZE = 100
OUTPUT_SIZE = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model = WordAVGModel(vocab_size=VOCAB_SIZE, 
                     embedding_size=EMBEDDING_SIZE, 
                     output_size=OUTPUT_SIZE,
                     pad_idx=PAD_IDX)

In [18]:
model

WordAVGModel(
  (embed): Embedding(25002, 100, padding_idx=1)
  (linear): Linear(in_features=100, out_features=1, bias=True)
)

In [19]:
# 查看模型有多少参数,参数太多容易过拟合
def count_parameters(model):
    # requires_grad:模型可以被训练
    # numel 帮你数一共有多少个参数
    return sum( p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(model)

2500301

glove初始化模型

In [20]:
pretrained_embedding = TEXT.vocab.vectors
pretrained_embedding.shape

torch.Size([25002, 100])

In [21]:
# 以_结尾的function都是inplace的操作，所以weight.data会被更新
model.embed.weight.data.copy_(pretrained_embedding)

model.embed.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_SIZE)
model.embed.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_SIZE)

### 训练模型


In [22]:
optimizer = torch.optim.Adam(model.parameters())

# BCE:binary crossEntropy 针对二分类的问题
# WithLogitsLoss:拿到的linear的数字可以是任何数字，不需要做sigmoid
crit = nn.BCEWithLogitsLoss()

model = model.to(device)
crit = crit.to(device)

计算预测的准确率

In [23]:
def binary_accuracy(preds, y):
    rounted_preds =  torch.round(torch.sigmoid(preds))
    correct = (rounted_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [24]:
def train(model, iterator, optimizer, crit):
    epoch_loss, epoch_acc = 0., 0.
    model.train()
    total_len = 0.
    for batch in iterator:
        preds = model(batch.text).squeeze()
        loss = crit(preds, batch.label)
        acc = binary_accuracy(preds, batch.label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item() * len(batch.label)
        epoch_acc += acc.item() * len(batch.label)
        total_len += len(batch.label)
        
    return epoch_loss / total_len, epoch_acc / total_len

In [25]:
def evaluate(model, iterator, crit):
    epoch_loss, epoch_acc = 0., 0.
    model.eval()
    total_len = 0.
    for batch in iterator:
        preds = model(batch.text).squeeze() # [batch_size, 1]
        loss = crit(preds, batch.label)
        acc = binary_accuracy(preds, batch.label)
        

        epoch_loss += loss.item() * len(batch.label)
        epoch_acc += acc.item() * len(batch.label)
        total_len += len(batch.label)
    model.train()
    
    
    return epoch_loss / total_len, epoch_acc / total_len

In [26]:
N_EPOCHS = 10
best_valid_acc = 0.
for epoch in range(N_EPOCHS):
    train_loss, train_acc = train(model, train_iterator, optimizer, crit)
    valid_loss, valid_acc = evaluate(model, valid_iterator, crit)
    
    
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        torch.save(model.state_dict(), "wordavg-model.pth")
    
    print("Epoch", epoch, "Train Loss", train_loss, "Train Acc", train_acc)
    print("Epoch", epoch, "Valid Loss", valid_loss, "Valid Acc", valid_acc)
        
        

Epoch 0 Train Loss 0.6001635191951479 Train Acc 0.7271428571428571
Epoch 0 Valid Loss 0.39377079436381657 Valid Acc 0.8250666666666666
Epoch 1 Train Loss 0.3918510187659945 Train Acc 0.8684571428571428
Epoch 1 Valid Loss 0.3197916497609268 Valid Acc 0.876
Epoch 2 Train Loss 0.293313466899097 Train Acc 0.9038285714285714
Epoch 2 Valid Loss 0.3346381409619624 Valid Acc 0.8888
Epoch 3 Train Loss 0.23977106667820897 Train Acc 0.9229142857142857
Epoch 3 Valid Loss 0.34886806211177995 Valid Acc 0.8946666666666667
Epoch 4 Train Loss 0.20022021001167595 Train Acc 0.9386285714285715
Epoch 4 Valid Loss 0.3686698123744119 Valid Acc 0.8985333333333333
Epoch 5 Train Loss 0.17094880359271275 Train Acc 0.9504571428571429
Epoch 5 Valid Loss 0.38884140564132297 Valid Acc 0.8998666666666667
Epoch 6 Train Loss 0.14529234856844747 Train Acc 0.9590857142857143
Epoch 6 Valid Loss 0.41627454002354847 Valid Acc 0.8990666666666667
Epoch 7 Train Loss 0.12318757290645735 Train Acc 0.9684
Epoch 7 Valid Loss 0.448

In [27]:
model.load_state_dict(torch.load("wordavg-model.pth"))

<All keys matched successfully>

In [28]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_sentiment(sentence):
    tokenized = [ tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device) # seq_len
    tensor = tensor.unsqueeze(1) # seq_len * batch_size(是1)
    pred = torch.sigmoid(model(tensor))
    
    return pred.item()
    

In [29]:
predict_sentiment("This film is horrible!")

1.459556077777591e-29

In [30]:
predict_sentiment("This film is terrific!")

1.0

In [31]:
predict_sentiment("This film is terrible!")

1.102174118859291e-18

### RNN模型
- 下面我们尝试把模型换成RNN。RNN经常会被用来encode一个sequence

<center>$h_t$ = $RNN(x_t,h_{t-1})$</center>

- 我们使用最后一个hidden state h_r来表示整个句子，最后会返回一个整个句子的vector
- 然后我们把h_r通过一个线性变换f，然后来预测句子的情感

![](images2\73.png)
![](images2\74.png)


In [32]:
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_size, output_size, pad_idx, hidden_size, dropout):
        super(RNNModel, self).__init__()
        self.embed = nn.Embedding(vocab_size, embedding_size, padding_idx = pad_idx)
        self.lstm = nn.LSTM(embedding_size, hidden_size)
        self.linear = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)
        
        
        
    def forward(self, text):
        embedded = self.embed(text) 
        embedded = self.dropout(embedded)
        
        # output是每一个output hidden是最后一个hidden
        output, (hidden,cell) = self.lstm(embedded)
        
        # hidden:2 * batch_size * hidden_size
        # cat:把两个tensor拼到一起,dim=1:把第一个维度hidden_size拼到一起
       # hidden = torch.cat([hidden[-1], hidden[-2]], dim=1)
        hidden = self.dropout(hidden.squeeze())
        
        return self.linear(hidden)
        

In [33]:
model =  RNNModel(vocab_size=VOCAB_SIZE, 
                  embedding_size=EMBEDDING_SIZE,
                  output_size=OUTPUT_SIZE,
                  pad_idx=PAD_IDX,
                  hidden_size=100,
                  dropout=0.5)

glove初始化模型

In [34]:
pretrained_embedding = TEXT.vocab.vectors
model.embed.weight.data.copy_(pretrained_embedding)

model.embed.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_SIZE)
model.embed.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_SIZE)

In [35]:
optimizer = torch.optim.Adam(model.parameters())
crit = nn.BCEWithLogitsLoss()

model = model.to(device)
crit = crit.to(device)

In [36]:
N_EPOCHS = 10
best_valid_acc = 0.
for epoch in range(N_EPOCHS):
    train_loss, train_acc = train(model, train_iterator, optimizer, crit)
    valid_loss, valid_acc = evaluate(model, valid_iterator, crit)
    
    
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        torch.save(model.state_dict(), "lstm-model.pth")
    
    print("Epoch", epoch, "Train Loss", train_loss, "Train Acc", train_acc)
    print("Epoch", epoch, "Valid Loss", valid_loss, "Valid Acc", valid_acc)
        
        

Epoch 0 Train Loss 0.6933248406546456 Train Acc 0.5025714285714286
Epoch 0 Valid Loss 0.6699832369168599 Valid Acc 0.5989333333333333
Epoch 1 Train Loss 0.6872026329857962 Train Acc 0.5174285714285715
Epoch 1 Valid Loss 0.6299762776692708 Valid Acc 0.6521333333333333
Epoch 2 Train Loss 0.684965097454616 Train Acc 0.5297714285714286
Epoch 2 Valid Loss 0.5954236539840698 Valid Acc 0.6808
Epoch 3 Train Loss 0.6733502835954939 Train Acc 0.5344571428571429
Epoch 3 Valid Loss 0.5147872895240784 Valid Acc 0.7536
Epoch 4 Train Loss 0.5424034136363438 Train Acc 0.7448
Epoch 4 Valid Loss 0.4002909813920657 Valid Acc 0.84
Epoch 5 Train Loss 0.4511684753043311 Train Acc 0.8149142857142857
Epoch 5 Valid Loss 0.4693680318832397 Valid Acc 0.8145333333333333
Epoch 6 Train Loss 0.42480001869542255 Train Acc 0.8229142857142857
Epoch 6 Valid Loss 0.3295076567351818 Valid Acc 0.8698666666666667
Epoch 7 Train Loss 0.37875672563399587 Train Acc 0.8478857142857142
Epoch 7 Valid Loss 0.3361778323471546 Valid 

### CNN模型

![](images2\75.png)

有若干个filter，每一个window都会返回若干个数字，channel number of filters 
![](images2\76.png)

![](images2\77.png)

维度从n * k --> (n-h+1) * num_filters --> 1 * num_filters(去掉1，num_filters就可以表示句子）

![](images2\78.png)

In [46]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_size, output_size, pad_idx, num_filters, filter_size, dropout):
        super(CNN, self).__init__()
        self.embed = nn.Embedding(vocab_size, embedding_size, padding_idx = pad_idx)
        self.conv = nn.Conv2d(in_channels = 1, out_channels = num_filters, kernel_size=(filter_size, embedding_size))
        self.linear = nn.Linear(embedding_size, output_size)
        self.dropout = nn.Dropout(dropout)
        
        
        
    def forward(self, text):
        # 和doc吻合
        text = text.permute(1, 0) # [batch_size, seq_len]
        embedded = self.embed(text) # [batch_size, seq_len, embedding_size]
        embedded = embedded.unsqueeze(1) # [batch_size, 1, seq_len, embedding_size]
         
        conved = F.relu(self.conv(embedded)) # [batch_size, num_filters, seq_len-filter_size+1, 1]
        conved = conved.squeeze(3) # [batch_size, num_filters, seq_len-filter_size+1]
        # max over time pooling
        pooled = F.max_pool1d(conved, conved.shape[2]) # [batch_size, num_filters, 1]
        pooled = pooled.squeeze(2)
        
        pooled= self.dropout(pooled)
        
        
        return self.linear(pooled)
        
        

In [47]:
model = CNN(vocab_size=VOCAB_SIZE, 
           embedding_size=EMBEDDING_SIZE,
           output_size=OUTPUT_SIZE,
           pad_idx=PAD_IDX,
           num_filters=100,
           filter_size = 3,
           dropout=0.5
           )
pretrained_embedding = TEXT.vocab.vectors
model.embed.weight.data.copy_(pretrained_embedding)

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embed.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_SIZE)
model.embed.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_SIZE)

optimizer = torch.optim.Adam(model.parameters())
crit = nn.BCEWithLogitsLoss()

model = model.to(device)
crit = crit.to(device)

In [49]:
N_EPOCHS = 10
best_valid_acc = 0.
for epoch in range(N_EPOCHS):
    train_loss, train_acc = train(model, train_iterator, optimizer, crit)
    valid_loss, valid_acc = evaluate(model, valid_iterator, crit)
    
    
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        torch.save(model.state_dict(), "cnn-model.pth")
    
    print("Epoch", epoch, "Train Loss", train_loss, "Train Acc", train_acc)
    print("Epoch", epoch, "Valid Loss", valid_loss, "Valid Acc", valid_acc)
        
        

Epoch 0 Train Loss 0.5922914501837322 Train Acc 0.6688
Epoch 0 Valid Loss 0.42673180599212646 Valid Acc 0.8101333333333334
Epoch 1 Train Loss 0.3936851459639413 Train Acc 0.8274285714285714
Epoch 1 Valid Loss 0.35699050936102866 Valid Acc 0.8402666666666667
Epoch 2 Train Loss 0.2797068047761917 Train Acc 0.888
Epoch 2 Valid Loss 0.33322876255313555 Valid Acc 0.8549333333333333
Epoch 3 Train Loss 0.207215193107192 Train Acc 0.9217714285714286
Epoch 3 Valid Loss 0.345472873754551 Valid Acc 0.8641333333333333
Epoch 4 Train Loss 0.15210083250754647 Train Acc 0.9421142857142857
Epoch 4 Valid Loss 0.3956528296339636 Valid Acc 0.8606666666666667
Epoch 5 Train Loss 0.10997888167521783 Train Acc 0.9611428571428572
Epoch 5 Valid Loss 0.4631723479911685 Valid Acc 0.8497333333333333
Epoch 6 Train Loss 0.08185167347971084 Train Acc 0.9712
Epoch 6 Valid Loss 0.5030428150888687 Valid Acc 0.8533333333333334
Epoch 7 Train Loss 0.06065377533397633 Train Acc 0.9797142857142858
Epoch 7 Valid Loss 0.608092

In [50]:
model.load_state_dict(torch.load("cnn-model.pth"))
test_loss, test_acc = evaluate(model, test_iterator, crit)
print("CNN model test loss: ", test_loss, "accuracy: ", test_acc)

CNN model test loss:  0.39394638749942185 accuracy:  0.84132
